In [16]:
import timeit
import random
import numpy

eps = 1e-16
N = 50000
locations = [0.0, 0.5, 1.0 - eps]


def median(x1, x2, x3):
    for a in range(7):
        if x1 <= x2 <= x3:
            return x2
        # Every loop I'm shufflin
        (x1, x2, x3) = (x2, x1, x3)
        if a % 2:
            (x1, x2, x3) = (x3, x1, x2)


def qsort_0(lst):
    indices = [(0, len(lst))]

    while indices:
        (frm, to) = indices.pop()
        if frm == to:
            continue

        # Find the partition:
        N = to - frm
        inds = [frm + int(N * n) for n in locations]
        values = [lst[ind] for ind in inds]
        partition = median(*values)

        # Split into lists:
        lower = []
        upper = []
        count = 0
        
        for item in lst[frm:to]:
            if item < partition:
                lower.append(item)
            elif item > partition:
                upper.append(item)
            else:
                count += 1

        ind1 = frm + len(lower)
        ind2 = ind1 + count

        # Push back into correct place:
        lst[frm:ind1] = lower
        lst[ind1:ind2] = [partition] * count
        lst[ind2:to] = upper

        # Enqueue other locations
        indices.append((frm, ind1))
        indices.append((ind2, to))
    return lst


def qsort_1(lst):
    """
    There is barely any performance advantage if all items are unique, 
    and we now pay the full price for a list of repeats, rather than running only once
    """
    indices = [(0, len(lst))]

    while indices:
        (frm, to) = indices.pop()
        if frm == to:
            continue

        # Find the partition:
        N = to - frm
        inds = [frm + int(N * n) for n in locations]
        values = [lst[ind] for ind in inds]
        partition = median(*values)

        # Split into lists:
        lower = [a for a in lst[frm:to] if a < partition]
        upper = [a for a in lst[frm:to] if a >= partition]

        ind1 = frm + len(lower)

        # Push back into correct place:
        lst[frm:ind1] = lower
        lst[ind1:to] = upper

        # Enqueue other locations
        indices.append((frm, ind1))
        indices.append((ind2, to))
    return lst


def qsort_2(lst):
    """
    Using the first item is akin to using the last item - on expectation exactly the same, 
    but much more prone to worst-case behaviors
    """
    indices = [(0, len(lst))]

    while indices:
        (frm, to) = indices.pop()
        if frm == to:
            continue

        # Find the partition:
        partition = lst[frm]

        # Split into lists:
        lower = [a for a in lst[frm:to] if a < partition]
        upper = [a for a in lst[frm:to] if a > partition]
        counts = sum([1 for a in lst[frm:to] if a == partition])

        ind1 = frm + len(lower)
        ind2 = ind1 + counts

        # Push back into correct place:
        lst[frm:ind1] = lower
        lst[ind1:ind2] = [partition] * counts
        lst[ind2:to] = upper

        # Enqueue other locations
        indices.append((frm, ind1))
        indices.append((ind2, to))
    return lst


def qsort_3(lst, frm=0, to=None):
    """
    In the worst case, the recurrence is T(n) = T(n-1) + T(0) + O(n),
    requiring a recursive depth of n.
    
    In the best case, the recurrence is T(n) = 2 T(n/2),
    requiring a recursive depth of log_2(n). 
    
    Thus, in the optimal case, we can sort lists of an absurd size (~ 2 ^ 500).
    """
    if to is None:
        to = len(lst)

    if frm == to:
        return

    # Find the partition:
    N = to - frm
    inds = [frm + int(N * n) for n in locations]
    values = [lst[ind] for ind in inds]
    partition = median(*values)

    # Split into lists:
    lower = [a for a in lst[frm:to] if a < partition]
    upper = [a for a in lst[frm:to] if a > partition]
    counts = sum([1 for a in lst[frm:to] if a == partition])

    ind1 = frm + len(lower)
    ind2 = ind1 + counts

    # Push back into correct place:
    lst[frm:ind1] = lower
    lst[ind1:ind2] = [partition] * counts
    lst[ind2:to] = upper

    # Enqueue other locations
    qsort_3(frm, ind1)
    qsort_3(ind2, to)
    
    return lst


def randomized_quicksort():
    lst = range(N)
    random.shuffle(lst)
    return qsort_0(lst)


def randomized_quicksort_no_median():
    lst = range(N)
    random.shuffle(lst)
    return qsort_2(lst)


def test_quicksort():
    lst = randomized_quicksort()
    assert (lst == range(N))


# Is our algorithm correct
test_quicksort()

# How fast is our algorithm
print timeit.timeit(randomized_quicksort, number=1)
print timeit.timeit(randomized_quicksort_no_median, number=1)

0.387264966965
0.325127840042
